# Load libraries

In [ ]:
rm(list=ls())
#### Packages installation
library(dplyr)
library(tidyverse)

# Read the input data
This input data set contains the 356,133 patients that met the inclusion criteria for our study. 
It contains one patient per row, and in the columns, it contains the main demographic information, followed by the SUD, marked as 1 when the patient had at least 1 ICD code for the category, and 0 when the patient had no ICD code in that category in the study period. 

In [ ]:
### Read input data
dataForRegression <- read.delim("demographic_icd_ML_aditional_f_column.txt", sep=",", header = TRUE) 
dim(dataForRegression)
#head(dataForRegression)


## Reformat the data for the model
We reformat the data, to present:
- Hispanic and Sex as 0-1
- Age as numeric
- FPL as character

We also rename the columns from th F codes to the actual names. 

In [ ]:
dataForRegression <- dataForRegression %>%
            dplyr::mutate( Hispanic = ifelse( Hispanic =="Y", 1, 0), 
                           Sex = ifelse( Sex =="M", 1, 0), 
                           Age = as.numeric( Age ), 
                           FPL = as.character(FPL )) %>%
            dplyr::select( -X, -Race, -M_Status, -State, -predicted_f_code  )

colnames( dataForRegression )[7:16] <- c("Alcohol","Opioid","Cannabis","Sedative_hypnoti_anxiolytic","Cocaine",
"OtherStimulant","Hallucinogen","NicotineDependence","Inhalant","Other_psychoactive_substance")

#head(dataForRegression)


## Regression analysis
We do a for loop that allow us to do a regression analysis for each of the SUDs. 
We use the formula: **sud ~ Sex + Age + Hispanic + FPL + Lang**

In [ ]:
# List of SUDs
sud_list <- c("Alcohol","Opioid","Cannabis","Sedative_hypnoti_anxiolytic","Cocaine",
"OtherStimulant","Hallucinogen","NicotineDependence","Inhalant","Other_psychoactive_substance") 

# Loop through each SUD
for (sud in sud_list) {
  print(sud)
  formula <- as.formula(paste(sud, "~ Sex + Age + Hispanic + FPL + Lang"))
  model <- glm(formula, data = dataForRegression, family = binomial)
  print( summary(model))
  print("####")
}


We have really low number of patients with Hallucinogen and Inhalant, that's why the model does not converge.  
We check below the number of cases and controls for each SUD. 

In [ ]:
summary(as.factor( dataForRegression$Alcohol ))
summary(as.factor( dataForRegression$Opioid ))
summary(as.factor( dataForRegression$Cannabis ))
summary(as.factor( dataForRegression$Sedative_hypnoti_anxiolytic ))
summary(as.factor( dataForRegression$Cocaine ))
summary(as.factor( dataForRegression$OtherStimulant ))
summary(as.factor( dataForRegression$Hallucinogen ))
summary(as.factor( dataForRegression$NicotineDependence ))
summary(as.factor( dataForRegression$Inhalant ))
summary(as.factor( dataForRegression$Other_psychoactive_substance ))


## Save the results in a table
We extract from each regression, the p-values and coefficients, and estimate the odds ratio and adjusted p-values. 

In [ ]:
# save results in a data.frame

# Create an empty list to store results
results_list <- list()

# Loop through each SUD
for (sud in sud_list) {
  formula <- as.formula(paste(sud, "~ Sex + Age + Hispanic + FPL + Lang"))
  model <- glm(formula, data = dataForRegression, family = binomial)
  results <- summary(model)
  
  # Extract relevant information from summary
  coefficients <- round(coef(results), 3)
  p_values <- round(coef(summary(model))[, "Pr(>|z|)"], 3)
  
  # Apply multiple testing correction (Benjamini-Hochberg)
  adjusted_p_values <- p.adjust(p_values, method = "BH")
  
  # Combine results into a data frame
  result_df <- data.frame(
    Predictor = rownames(coefficients),
    Coefficient = coefficients,
    Odds_Ratio = round(exp(coefficients),3),
    P_value = p_values, 
    Adjusted_P_Value = round(adjusted_p_values,3),
    stringsAsFactors = FALSE
  )
  
  # Add result_df to the results_list
  results_list[[sud]] <- result_df
}

# Combine results for all SUDs into one data frame
all_results_df <- do.call(rbind, results_list)

write.csv(all_results_df, file = "./regression_analysis_results_fullPopulation.csv", row.names = FALSE)



## Regression analysis for the hispanic subpopulation

In [ ]:
dataForRegressionHispanic <- dataForRegression %>%
            dplyr::filter(Hispanic == 1 )

In [ ]:
# Loop through each SUD
for (sud in sud_list) {
  print(sud)
  formula <- as.formula(paste(sud, "~ Sex + Age + + FPL + Lang"))
  model <- glm(formula, data = dataForRegressionHispanic, family = binomial)
  print( summary(model))
  print("####")
}

In [ ]:
# save results in a data.frame
rm(results_list)
# Create an empty list to store results
results_list <- list()

# Loop through each SUD
for (sud in sud_list) {
  formula <- as.formula(paste(sud, "~ Sex + Age + FPL + Lang"))
  model <- glm(formula, data = dataForRegressionHispanic, family = binomial)
  results <- summary(model)
  
  # Extract relevant information from summary
  coefficients <- round(coef(results), 3)
  p_values <- round(coef(summary(model))[, "Pr(>|z|)"], 3)
  
  # Apply multiple testing correction (Benjamini-Hochberg)
  adjusted_p_values <- p.adjust(p_values, method = "BH")
  
  # Combine results into a data frame
  result_df <- data.frame(
    Predictor = rownames(coefficients),
    Coefficient = coefficients,
    Odds_Ratio = round(exp(coefficients),3),
    P_value = p_values, 
    Adjusted_P_Value = round(adjusted_p_values,3),
    stringsAsFactors = FALSE
  )
  
  # Add result_df to the results_list
  results_list[[sud]] <- result_df
}

# Combine results for all SUDs into one data frame
hispanic_results_df <- do.call(rbind, results_list)

write.csv(hispanic_results_df, file = "./regression_analysis_results_hispanicPopulation.csv", row.names = FALSE)

## Regression analysis for the non-hispanic subpopulation

In [ ]:
dataForRegressionNoHispanic <- dataForRegression %>%
            dplyr::filter(Hispanic == 0 )

In [ ]:
for (sud in sud_list) {
  print(sud)
  formula <- as.formula(paste(sud, "~ Sex + Age + + FPL + Lang"))
  model <- glm(formula, data = dataForRegressionNoHispanic, family = binomial)
  print( summary(model))
  print("####")
}

In [ ]:
# save results in a data.frame
rm(results_list)
# Create an empty list to store results
results_list <- list()

# Loop through each SUD
for (sud in sud_list) {
  formula <- as.formula(paste(sud, "~ Sex + Age + FPL + Lang"))
  model <- glm(formula, data = dataForRegressionNoHispanic, family = binomial)
  results <- summary(model)
  
  # Extract relevant information from summary
  coefficients <- round(coef(results), 3)
  p_values <- round(coef(summary(model))[, "Pr(>|z|)"], 3)
  
  # Apply multiple testing correction (Benjamini-Hochberg)
  adjusted_p_values <- p.adjust(p_values, method = "BH")
  
  # Combine results into a data frame
  result_df <- data.frame(
    Predictor = rownames(coefficients),
    Coefficient = coefficients,
    Odds_Ratio = round(exp(coefficients),3),
    P_value = p_values, 
    Adjusted_P_Value = round(adjusted_p_values,3),
    stringsAsFactors = FALSE
  )
  
  # Add result_df to the results_list
  results_list[[sud]] <- result_df
}

# Combine results for all SUDs into one data frame
non_hispanic_results_df <- do.call(rbind, results_list)

write.csv(non_hispanic_results_df, file = "./regression_analysis_results_non_hispanicPopulation.csv", row.names = FALSE)

In [ ]:
sessionInfo()